In [ ]:
Following code creates a database named "StackOverflow" and a container named "Posts"
It inserts 10k documents into the container.
You may need to run this code to execute C# codes in different folders.

In [ ]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Net.Http;
using Newtonsoft.Json;
using System.Collections.ObjectModel;

Following connection string is from Azure Cosmos DB Emulator.
Download it and set it up if you like to use this locally.
It uses Bulk feature to insert operations faster.

In [ ]:
var cstring = "AccountEndpoint=https://localhost:8081/;AccountKey=C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
client = new CosmosClient(cstring, new CosmosClientOptions() {AllowBulkExecution = true});
var newdb = await client.CreateDatabaseIfNotExistsAsync("StackOverflow");
ContainerProperties props = new ContainerProperties("Posts0", "/OwnerUserId");
var postContainer = await db.CreateContainerIfNotExistsAsync(props, throughput: 4000);


Following is the data model we will store in the container.

In [ ]:
public class Post    
{
    public string id { get; set; }
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int AnswerCount { get; set; }
    public int CommentCount { get; set; }
    public int FavoriteCount { get; set; }
    public int AcceptedAnswerId { get; set; }
    public DateTime? CreatedOn { get; set; }
    public DateTime? ClosedDate { get; set; }
    public int OwnerUserId { get; set; }
    public string OwnerDisplayName { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public string Tags { get; set; }
    public float[] bodyvector {get;set;}
}

In [ ]:
using Newtonsoft.Json.Linq;

// It takes 15 minutes to run this code
var json = await new HttpClient().GetStringAsync("https://raw.githubusercontent.com/hsavran/Presentations/refs/heads/main/stackoverflow.json");
var postList = JsonConvert.DeserializeObject<List<Post>>(json);
postList.Count.Display(); // Display the number of posts
//write the post to the container in bulk mode
var postContainer = db.GetContainer("Posts");
foreach (var post in postList)
{        
    var response = await postContainer.CreateItemAsync(post, new PartitionKey(post.OwnerUserId));
    response.StatusCode.Display(); // Display the status code of the response
}